In [130]:
import ee
import folium

service_account = 'gee-service-account@gee-experimentation.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'private-key.json')
ee.Initialize(credentials)

In [135]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

In [144]:
# Define the center of our map.
lat, lon = 48.32, 2.455

poi = ee.Geometry.Point(lon, lat)

# Import the MODIS land cover collection.
lc = ee.ImageCollection('MODIS/006/MCD12Q1')

# Import the MODIS land surface temperature collection.
lst = ee.ImageCollection('MODIS/006/MOD11A1')

# Import the USGS ground elevation image.
elv = ee.Image('USGS/SRTMGL1_003')

# Initial date of interest (inclusive).
i_date = '2010-01-01'
# Final date of interest (exclusive).
f_date = '2020-01-02'

# Make pixels with elevation below sea level transparent.
elv_img = elv.updateMask(elv.gt(0))

# Define a region of interest with a buffer zone of 1000 km around lat and long.
roi = poi.buffer(1e6)

# Select a specific band and dates for land cover.
lc_img = lc.select('LC_Type1').filterDate(i_date).first()

# Selection of appropriate bands and dates for LST.
lst = lst.select('LST_Day_1km', 'QC_Day').filterDate(i_date, f_date)

# Reduce the LST collection by mean.
lst_img = lst.mean()

# Reduce the LST collection by mean.
lst_img = lst.mean()

# Adjust for scale factor.
lst_img = lst_img.select('LST_Day_1km').multiply(0.02)

# Convert Kelvin to Celsius.
lst_img = lst_img.select('LST_Day_1km').add(-273.15)

# Set visualization parameters for land cover.
lc_vis_params = {
    'min': 1,'max': 17,
    'palette': ['05450a','086a10', '54a708', '78d203', '009900', 'c6b044',
                'dcd159', 'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44',
                'a5a5a5', 'ff6d4c', '69fff8', 'f9ffa4', '1c0dff'],
}

# Set visualization parameters for ground elevation.
elv_vis_params = {
    'min': 0, 'max': 8000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

# Set visualization parameters for land surface temperature.
lst_vis_params = {
    'min': -40, 'max': 50,
    'palette': ['white', 'blue', 'green', 'yellow', 'orange', 'red'],
}

ee_tiles = [elv_img, lst_img, lc_img]
ee_vis_params = [elv_vis_params, lst_vis_params, lc_vis_params]
ee_tiles_names = ['Elevation', 'Land Surface Temperature', 'Land Cover']

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer
my_map = folium.Map(location=[lat, lon], zoom_start=7)

for tile, vis_param, name in zip(ee_tiles, ee_vis_params, ee_tiles_names):
    my_map.add_ee_layer(tile, vis_param, name)
folium.LayerControl(collapsed = False).add_to(my_map)

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)